In [2]:
# -*- coding: utf-8 -*-
"""rise_spatial_multiplicative_norm_zero_cineca.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1-4UndkKCTVbDRQZql3km54NZicEG05gb

### ***Cineca***
"""

import tensorflow as tf
#from tensorflow.keras import layers, activations, callbacks, models
import numpy as np
import pickle
import os
from keras.models import load_model
from skimage.transform import resize
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
# Save Execution Time
import datetime
import sys

"""
##### ***Data & Black-Box***

"""

RESULT_DIR = str(sys.argv[1])
print(f"RESULT_DIR: {RESULT_DIR}")

# IMPORTO I DATI PER VOTTIGNASCO
import os

# Ottieni il percorso effettivo da una variabile d'ambiente
work_path = os.environ['WORK']  # Ottieni il valore della variabile d'ambiente WORK
v_test_OHE_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_month_OHE.npy")
v_test_image_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_normalized_image_sequences.npy")
v_test_target_dates_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_target_dates.npy")
v_test_normalization_factors_std_path  = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/normalization_factors/Vottignasco_00425010001_training_target_std.npy")
v_test_normalization_factors_mean_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/normalization_factors/Vottignasco_00425010001_training_target_mean.npy")

# Carica l'array numpy dai file
vottignasco_test_OHE    = np.load(v_test_OHE_path)
vottignasco_test_image  = np.load(v_test_image_path)
vottignasco_test_dates  = np.load(v_test_target_dates_path)
vott_target_test_std    = np.load(v_test_normalization_factors_std_path) 
vott_target_test_mean   = np.load(v_test_normalization_factors_mean_path)


print(len(vottignasco_test_dates))
print(len(vottignasco_test_image))
print(len(vottignasco_test_OHE))

#print(vottingasco_test_OHE[0], "\n")
#print(vottignasco_test_image[0][0], "\n")

# """##### ***Black Boxes***""

# Se vuoi abilitare il dropout a runtime
mc_dropout = True

# Definizione della classe personalizzata doprout_custom
class doprout_custom(tf.keras.layers.SpatialDropout1D):
    def call(self, inputs, training=None):
        if mc_dropout:
            return super().call(inputs, training=True)
        else:
            return super().call(inputs, training=False)

# Percorso della directory su Cineca
base_dir = os.path.join(os.environ['WORK'], "Water_Resources/rise-video/trained_models/seq2val/Vottignasco")
lstm_suffix = 'time_dist_LSTM'

vott_lstm_models = []

def extract_index(filename):
    """Funzione per estrarre l'indice finale dal nome del file."""
    return int(filename.split('_LSTM_')[-1].split('.')[0])

# Trova tutti i file .keras nella cartella e li aggiunge alla lista
for filename in os.listdir(base_dir):
    if lstm_suffix in filename and filename.endswith(".keras"):
        vott_lstm_models.append(os.path.join(base_dir, filename))

# Ordina i modelli in base all'indice finale
vott_lstm_models = sorted(vott_lstm_models, key=lambda x: extract_index(os.path.basename(x)))

# Lista per i modelli caricati
vott_lstm_models_loaded = []

for i, model_lstm_path in enumerate(vott_lstm_models[:10]):  # Prendo i primi 10 modelli ordinati
    #print(f"Caricamento del modello LSTM {i+1}: {model_lstm_path}")

    # Carico il modello con la classe custom
    model = load_model(model_lstm_path, custom_objects={"doprout_custom": doprout_custom})

    # Aggiungo il modello alla lista
    vott_lstm_models_loaded.append(model)

print(vott_lstm_models_loaded)

"""### ***RISE-Spatio_Temporal***

#### ***Generation Masks (3D): Uniforme***
"""

import numpy as np
from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm

def generate_masks_3d(N, input_size, seed=42, **kwargs):
    """
    Parametri:
    - input_size: (t, h, w) -> dimensione finale della maschera 3D (time, Height, Width)
    """
    t, h, w = input_size
    l = kwargs.get("l", 8)   # Dimensione della small mask per il tempo
    s = kwargs.get("s", 8)   # Fattore per downsampling spaziale (per h e w)
    p1 = kwargs.get("p1", 0.4)  # Probabilità di attivazione

    np.random.seed(seed)

    # Calcola le dimensioni spaziali della small mask usando solo h e w
    small_s = np.ceil(np.array([h, w]) / s).astype(int)

    # Genera una maschera 3D casuale di dimensione (l, small_h, small_w)
    grid = np.random.rand(N, l, small_s[0], small_s[1]) < p1
    grid = grid.astype('float32')

    # Struttura per le maschere finali di dimensione (N, t, h, w)
    masks = np.empty((N, t, h, w))

    # Coordinate per l'interpolazione spaziale
    grid_x = np.linspace(0, small_s[0] - 1, small_s[0])
    grid_y = np.linspace(0, small_s[1] - 1, small_s[1])
    grid_t = np.linspace(0, l - 1, l)

    for i in tqdm(range(N), desc='Generating masks'):
      # Crea un interpolatore per la maschera corrente
      interpolator = RegularGridInterpolator(
        (grid_t, grid_x, grid_y), grid[i], method='linear', bounds_error=False, fill_value=0
      )
      new_t = np.linspace(0, l - 1, t)
      new_x = np.linspace(0, small_s[0] - 1, h)
      new_y = np.linspace(0, small_s[1] - 1, w)
      mesh_t, mesh_x, mesh_y = np.meshgrid(new_t, new_x, new_y, indexing='ij')
      points = np.stack((mesh_t, mesh_x, mesh_y), axis=-1).reshape(-1, 3)
      interpolated_mask = interpolator(points)
      masks[i] = interpolated_mask.reshape(t, h, w)


    masks = masks[~(masks == 0).all(axis=(1, 2, 3))]
    return masks

"""#### ***Application Masks***"""

def multiplicative_uniform_noise_onechannel(images, masks, channel, **kwargs):
    """
    Applica noise moltiplicativo continuo in 3D alle immagini sulla base delle maschere fornite.

    Parametri:
    - images: array di shape (104, 5, 8, C) -> serie temporale di immagini con canali multipli
    - masks: array di shape (N, 104, 5, 8) -> maschere continue con valori [0, 1]
    - channel: canale specifico su cui applicare il noise (0: Prec, 1: TMax, 2: TMin)
    - std_zero_value: valore del disturbo da applicare dove mask = 0
    """
    std_zero_value = kwargs.get("std_zero_value", -0.6486319166678826)

    masked = []

    # Itera su tutte le N maschere generate
    for mask in masks:
        masked_images = copy.deepcopy(images)  # Copia profonda delle immagini originali

        # Estrai solo il canale desiderato
        channel_values = masked_images[..., channel]

        # Applica la formula: v(p) + z (1-p)
        perturbed_values = channel_values * mask + (1 - mask) * std_zero_value

        # Sovrascrivi il canale con i valori perturbati
        masked_images[..., channel] = perturbed_values

        masked.append(masked_images)

    return masked

def ensemble_predict(models, images, x3_exp, batch_size=1000):
    # Assicuriamoci che images sia una lista
    if not isinstance(images, list):
        images = [images]

    len_x3 = len(images)

    # Convertiamo x3_exp in un tensore replicato per ogni immagine
    x3_exp_tensor = tf.convert_to_tensor(x3_exp, dtype=tf.float32)

    # Lista per raccogliere le predizioni finali
    final_preds = []

    # Processamento a batch
    for i in range(0, len_x3, batch_size):
        batch_images = images[i:i + batch_size]
        batch_len = len(batch_images)

        # Conversione batch in tensori
        Y_test = tf.stack([tf.convert_to_tensor(img, dtype=tf.float32) for img in batch_images])
        Y_test_x3 = tf.tile(tf.expand_dims(x3_exp_tensor, axis=0), [batch_len, 1, 1])

        # Raccoglie le predizioni di tutti i modelli per il batch corrente
        batch_preds = []

        for model in models:
            preds = model.predict([Y_test, Y_test_x3], verbose=0)
            batch_preds.append(preds)

        # Converte le predizioni del batch in un tensore e calcola la media
        batch_preds_tensor = tf.stack(batch_preds)
        mean_batch_preds = tf.reduce_mean(batch_preds_tensor, axis=0)

        # Aggiunge le predizioni del batch alla lista finale
        final_preds.extend(mean_batch_preds.numpy())

    return np.array(final_preds)

"""#### ***Saliency Map***"""

def calculate_saliency_map(preds_array, masks):
    """
    Calcola la mappa di salienza media data una serie di predizioni e maschere.

    :param preds_array: Array di predizioni (numero di maschere x dimensioni predizione).
    :param masks: Array di maschere (numero di maschere x dimensioni maschera).
    :return: Mappa di salienza media.
    """
    sal = []
    for j in range(len(masks)):
        sal_i = preds_array[j] * np.abs(masks[j])
        sal.append(sal_i.reshape(-1, 5, 8))  # Adatta la shape secondo il formato orginiale dei frame

    # Rimuovere le dimensioni extra per fare np.mean lungo axis=0. masks ha shape (N,5,8,1)
    masks_squeezed = np.squeeze(np.abs(masks))
    # Ora calcola la media lungo l'asse 0
    ev_masks = np.mean(masks_squeezed, axis=0)
    ev_masks[ev_masks == 0] = 1e-8  # Sostituiamo gli zeri con un piccolo valore

    sal = (1/ev_masks) * np.mean(sal, axis=0)

    return sal

"""#### ***Spatio_Temporal-RISE: Framework***"""

def rise_st_explain_diff_pred(nr_instance, data_test_image, data_test_OHE, models, channel,
                                   N, generate_masks_fn, seed, perturb_instance_fn, calculate_saliency_map_fn, H_station=390.0, **kwargs):
  print(f"############################### RISE-Temporal on Instance #{nr_instance} ###########################")
  instance    = copy.deepcopy(data_test_image[nr_instance])
  x3_instance = copy.deepcopy(data_test_OHE[nr_instance])

  input_size = (instance.shape[0], instance.shape[1], instance.shape[2])

  masks = generate_masks_fn(N, input_size, seed, **kwargs)
  perturbed_instances = perturb_instance_fn(instance, masks, channel)

  # Predizione su Istanza Originale
  pred_original = ensemble_predict(models, instance, x3_instance)
  # Predizioni su Istanze Perturbate
  preds_masked = ensemble_predict(models, perturbed_instances, x3_instance)
  # Denormalizzazione con H_station (altezza della stazione)
  denorm_pred_original = pred_original * vott_target_test_std + vott_target_test_mean
  denorm_preds_masked  = [pred_masked * vott_target_test_std + vott_target_test_mean for pred_masked in preds_masked]
  denormalized_H_pred_original = H_station - denorm_pred_original
  denormalized_H_preds_masked  = [H_station - denorm_pred_masked for denorm_pred_masked in denorm_preds_masked]

  # Differenza tra predizione originale e perturbata
  diff_pred = [denormalized_H_pred_original - denorm_H_pred for denorm_H_pred in denormalized_H_preds_masked]
  weights = np.concatenate(diff_pred, axis=0)

  # Calcolo della mappa di salienza
  saliency_map_i = calculate_saliency_map_fn(weights, masks)
  print(f"############### Processo completato. Mappa di salienza generata per Istanza #{nr_instance} ###############")
  return np.squeeze(saliency_map_i)

"""#### ***Evaluation Metrics***"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

def calculate_auc(x, y):
    """
    Calcola l'area sotto la curva (AUC) utilizzando il metodo del trapezio.

    :param x: Valori dell'asse x (frazione dei pixel/frame inseriti).
    :param y: Valori dell'asse y (errori calcolati).
    :return: Area sotto la curva.
    """
    return np.trapz(y, x)

import numpy as np

def get_flatten_saliency_video_ordered_by_importance(saliency_video):
    """
    Riceve un array 3D (T, H, W) con i valori di salienza e restituisce
    una lista ordinata dei pixel per importanza.

    :param saliency_video: np.array di shape (T, H, W) con i valori di salienza.
    :return: Lista ordinata di tuple (salienza, [t, x, y]), con la salienza decrescente.
    """
    # Flatten dei valori di salienza
    flatten_saliency_video = saliency_video.flatten()

    # Ottieni gli indici originali [t, x, y] per ogni pixel
    indices = np.array(np.unravel_index(np.arange(flatten_saliency_video.size), saliency_video.shape)).T

    # Combina valori e indici: lista di tuple (salienza, [t, x, y])
    saliency_video_value_with_indices = list(zip(flatten_saliency_video, indices))

    # Ordina per valore di salienza (decrescente) e, a parità, per le coordinate (decrescente)
    sorted_saliency = sorted(
        saliency_video_value_with_indices,
        key=lambda x: (x[0], -x[1][0], -x[1][1], -x[1][2]),
        reverse=True
    )

    return sorted_saliency


def plot_insertion_curve(total_errors, auc, title="Insertion Metric Curve"):
    """
    Plotta la curva di metrica di insertion con l'errore medio quadratico.

    :param total_errors: Lista dei valori dell'errore per ogni frazione di pixel inseriti.
    :param auc: Valore dell'Area Under Curve (AUC) calcolato.
    :param title: Titolo del grafico (default: "Insertion Metric Curve").
    """

    # Nuovo asse X normalizzato tra 0 e 1
    x = np.linspace(0, 1, len(total_errors))

    # Plot della curva dell'errore e dell'area sotto la curva (AUC)
    plt.figure(figsize=(7, 5))
    plt.plot(x, total_errors, linestyle='-', color='blue')

    # Area sotto la curva
    plt.fill_between(x, total_errors, color='skyblue', alpha=0.4)

    # Testo "Error curve" in alto a dx con font più piccolo
    plt.legend(['Error curve'], loc='upper right', fontsize=9)

    # Testo AUC appena sotto "Error curve"
    plt.text(x[-1] - 0.02, max(total_errors) * 0.9,
             f'AUC: {auc:.2f}',
             horizontalalignment='right',
             fontsize=8,
             bbox=dict(facecolor='white', alpha=0.5))

    # Etichette assi
    plt.xlabel('Fraction of pixels inserted')
    plt.ylabel('Mean Squared Error')

    # Titolo del grafico
    plt.title(title)

    # Mostra il grafico
    plt.show()


def plot_deletion_curve(total_errors, auc, title="Deletion Metric Curve"):
    """
    Plotta la curva della metrica di deletion con l'errore medio quadratico.

    :param total_errors: Lista dei valori dell'errore per ogni frazione di pixel rimossi.
    :param auc: Valore dell'Area Under Curve (AUC) calcolato.
    :param title: Titolo del grafico (default: "Deletion Metric Curve").
    """

    # Normalizzazione dell'asse X tra 0 e 1
    x = np.linspace(0, 1, len(total_errors))

    # Creazione del plot
    plt.figure(figsize=(7,5))
    plt.plot(x, total_errors, linestyle='-', color='red')

    # Area sotto la curva
    plt.fill_between(x, total_errors, color='lightcoral', alpha=0.4)

    # Testo "Error curve" in alto a sx con font più piccolo
    plt.legend(['Error curve'], loc='lower right',  bbox_to_anchor=(0.97, 0.02))

    # Testo AUC leggermente spostato sotto la legenda
    plt.text(0.941, 0.13, f'AUC: {auc:.2f}',
             transform=plt.gca().transAxes,
             fontsize=10,
             verticalalignment='bottom',
             horizontalalignment='right',
             bbox=dict(facecolor='white', alpha=0.6, edgecolor='grey'))

    # Etichette degli assi
    plt.xlabel('Fraction of pixels removed')
    plt.ylabel('Mean Squared Error')

    # Titolo del grafico
    plt.title(title)

    # Mostra il grafico
    plt.show()

"""##### ***Insertion***"""

def update_instance_with_pixels(current_instance, original_instance, t,x,y):
    """
    Aggiorna l'immagine inserendo i pixel più importanti.

    :param current_instance: Istanza corrente.
    :param original_instance: Istanza originale.
    :param t: coordinata t del pixel da inserire
    :param x: coordinata x del pixel da inserire
    :param y: coordinata y del pixel da inserire
    :return: Istanza aggiornata con il superpixel.
    """
    new_current_instance = current_instance.copy()
    new_current_instance[t, x, y, 0] = original_instance[t, x, y, 0]

    return new_current_instance


def insertion(model, original_instance, x3_instance, sorted_per_importance_pixels_index, initial_blurred_instance, original_prediction, H_station=390.0):
    """
    Calcola la metrica di inserimento per una spiegazione data.

    :param model: Black-box.
    :param original_instance: Istanza originale.
    :param sorted_per_importance_pixels_index: Lista di liste di tutti i superpixel per importanza
    :param initial_blurred_images: Immagine iniziale con tutti i pixel a zero.
    :return: Lista degli errori ad ogni passo di inserimento.
    """

    # Lista per memorizzare le istanze a cui aggiungo pixel mano a mano. Inizializzata con istanza iniziale blurrata
    insertion_images = [initial_blurred_instance]

    # Predizione sull'immagine iniziale (tutti i pixel a zero)
    I_prime = copy.deepcopy(initial_blurred_instance)

    # Aggiungere gradualmente i pixel (per ogni frame) più importanti. Ottengo una lista con tutte le img con i pixel aggiunti in maniera graduale
    for t,x,y in sorted_per_importance_pixels_index:
        I_prime = update_instance_with_pixels(I_prime, original_instance, t,x,y)
        insertion_images.append(I_prime)

    insertion_images = [img.astype(np.float32) for img in insertion_images]
    # Calcolo le predizioni sulle istanze a cui ho aggiunto i pixel in maniera graduale
    new_predictions = ensemble_predict(model, insertion_images, x3_instance)
    denorm_new_predictions  = [new_prediction * vott_target_test_std + vott_target_test_mean for new_prediction in new_predictions]
    denormalized_H_new_predictions  = [H_station - denorm_new_prediction for denorm_new_prediction in denorm_new_predictions]
    # Rispetto ad ogni suddetta predizione, calcolo il MSE rispetto la pred sull'istanza originaria (come da test-set). Ignora la prima che è sull'img blurrata originale
    errors = [mean_squared_error(original_prediction, masked_pred) for masked_pred in denormalized_H_new_predictions[1:]]

    initial_error = mean_squared_error(original_prediction, denormalized_H_new_predictions[0])
    print(f"Initial Prediction with Blurred Instance. Prediction: {denormalized_H_new_predictions[0]}, error: {initial_error}")
    
    only_inserted_pixel_new_predictions = denormalized_H_new_predictions[1:]
    for nr_pixel, error in enumerate(errors):
      print(f"Inserted Pixel: {sorted_per_importance_pixels_index[nr_pixel]}. Prediction: {only_inserted_pixel_new_predictions[nr_pixel]}, error: {error}")

    total_errors = [initial_error] + errors # Errore iniziale + errori su tutti i pixel inseriti

    # # Nuovo asse X
    x = np.linspace(0, 1, len(total_errors))
    # Calcolo dell'AUC con il nuovo asse x
    auc = calculate_auc(x, total_errors)
    print(f"Area under the curve (AUC): {auc}")
    return total_errors,auc

"""##### ***Deletion***"""

def update_image_by_removing_pixels(current_instance, t, x, y, std_zero_value=-0.6486319166678826):
    """
    Aggiorna l'immagine rimuovendo i pixel x,y indicati.

    :param current_instance: istanza corrente.
    :param t: coordinata t del pixel da rimuovere
    :param x: coordinata x del pixel da rimuovere
    :param y: coordinata y del pixel da rimuovere
    :return: Istanza aggiornata con x,y rimossi su tutti time-step.
    """
    new_instance = copy.deepcopy(current_instance)
    new_instance[t, x, y, 0] = std_zero_value # Imposta i pixel a zero normalizzato per Prec
    # new_instance[t, x, y, 1] = 0.0 # Imposta i pixel a zero per Tmax
    # new_instance[t, x, y, 2] = 0.0 # Imposta i pixel a zero per Tmin
    return new_instance


def deletion(models, original_instance, x3_instance, sorted_per_importance_pixels_index, original_prediction, H_station=390.0):
    """
    Calcola la metrica di rimozione per una spiegazione data.

    :param models: Lista di modelli pre-addestrati.
    :param original_instance: Immagine originale.
    :param x3_instance: Codifica one-hot per la previsione.
    :param sorted_per_importance_pixels_index: Indici dei pixel in ordine di importanza.
    :return: Lista degli errori, auc ad ogni passo di rimozione.
    """
    # Lista per memorizzare le img a cui elimino gradualmente i pixels (per ogni time-step)
    deletion_images = []

    # Inizializzazione
    I_prime = copy.deepcopy(original_instance)

    # Aggiungere gradualmente i pixel (per ogni frame) più importanti. Ottengo una lista con tutte le img con i pixel rimossi
    for t,x,y in sorted_per_importance_pixels_index:
        I_prime = update_image_by_removing_pixels(I_prime, t, x, y)
        deletion_images.append(I_prime)

    # Calcolo della predizione su tutte le img a cui ho rimosso gradualmente i pixel
    new_predictions = ensemble_predict(models, deletion_images, x3_instance)
    denorm_new_predictions  = [new_prediction * vott_target_test_std + vott_target_test_mean for new_prediction in new_predictions]
    denormalized_H_new_predictions  = [H_station - denorm_new_prediction for denorm_new_prediction in denorm_new_predictions]
    # Calcolo del mse rispetto la predizione originale
    errors = [mean_squared_error(original_prediction, masked_pred) for masked_pred in denormalized_H_new_predictions]

    initial_error = 0.0
    print(f"Initial Prediction with Original Images, prediction: {original_prediction}, error: {initial_error}")

    #for nr_pixel, error in enumerate(errors):
    #    print(f"Removed pixel {sorted_per_importance_pixels_index[nr_pixel]}, new prediction: {denormalized_H_new_predictions[nr_pixel]}, error: {error}")

    total_errors = [initial_error] + errors  # Errore iniziale + errori su tutti i pixel rimossi

    # Normalizzare la frazione di pixel rimossi
    x = np.linspace(0, 1, len(total_errors))
    # Calcolo dell'AUC
    auc = calculate_auc(x, total_errors)

    print(f"Area under the curve (AUC): {auc}")
    return total_errors, auc

2025-03-08 10:40:05.991641: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


RESULT_DIR: --f=/run/user/129071/jupyter/runtime/kernel-v31f6608d1519e8f10431900f964bc8368b45ffccc.json
105
105
105
[<keras.src.engine.functional.Functional object at 0x7f4942719ff0>, <keras.src.engine.functional.Functional object at 0x7f492bb12e90>, <keras.src.engine.functional.Functional object at 0x7f492b8c6800>, <keras.src.engine.functional.Functional object at 0x7f492b91f700>, <keras.src.engine.functional.Functional object at 0x7f492b967820>, <keras.src.engine.functional.Functional object at 0x7f492b84e3e0>, <keras.src.engine.functional.Functional object at 0x7f492b850250>, <keras.src.engine.functional.Functional object at 0x7f492b6c87f0>, <keras.src.engine.functional.Functional object at 0x7f492b8269b0>, <keras.src.engine.functional.Functional object at 0x7f492b76e9b0>]


In [3]:
nr_instance = 0
models = vott_lstm_models_loaded
channel_prec = 0

seed = 42

N = 2500

l = 12
s = 3
p = 0.5

time_start = datetime.datetime.now()
        
saliency_video_i = rise_st_explain_diff_pred(nr_instance, vottignasco_test_image, vottignasco_test_OHE, models, channel_prec,
                                                    N, generate_masks_3d, seed, multiplicative_uniform_noise_onechannel, calculate_saliency_map, l=l, s=s, p1=p)

time_end = datetime.datetime.now()
exec_time = time_end - time_start
            
print(exec_time)

############################### RISE-Temporal on Instance #0 ###########################


Generating masks: 100%|██████████| 2500/2500 [00:18<00:00, 132.82it/s]


############### Processo completato. Mappa di salienza generata per Istanza #0 ###############
0:01:08.628119


In [4]:
saliency_video_i

array([[[0.96366823, 0.96409571, 0.96451565, ..., 0.95923076,
         0.95528087, 0.95131824],
        [0.9617231 , 0.96208956, 0.96245229, ..., 0.95903714,
         0.9564154 , 0.95380414],
        [0.9598166 , 0.96010836, 0.9603993 , ..., 0.95884444,
         0.9575364 , 0.95624284],
        [0.9579476 , 0.95815163, 0.9583566 , ..., 0.95865267,
         0.95864409, 0.95863567],
        [0.95611499, 0.95621893, 0.95632411, ..., 0.95846182,
         0.95973873, 0.96098392]],

       [[0.96406442, 0.9644782 , 0.96488575, ..., 0.9598332 ,
         0.95606267, 0.95229037],
        [0.96191236, 0.96232332, 0.96273058, ..., 0.95946009,
         0.95689217, 0.95433624],
        [0.95980129, 0.96019647, 0.96059047, ..., 0.95908912,
         0.95771269, 0.95634965],
        [0.95773005, 0.95809711, 0.95846528, ..., 0.95872027,
         0.95852438, 0.95833138],
        [0.95569751, 0.95602472, 0.95635486, ..., 0.95835352,
         0.95932738, 0.96028216]],

       [[0.96445839, 0.96485955, 0.9

In [6]:
all_important_pixels = get_flatten_saliency_video_ordered_by_importance(saliency_video_i)[::-1]

all_important_pixels

[(0.8651279671661671, array([94,  4,  3])),
 (0.8652396120421357, array([94,  4,  4])),
 (0.8679284042503671, array([95,  4,  4])),
 (0.8679938909586757, array([95,  4,  3])),
 (0.868676425260485, array([94,  3,  4])),
 (0.8688209320166614, array([94,  3,  3])),
 (0.8705678318515784, array([93,  4,  3])),
 (0.870627131300203, array([96,  4,  4])),
 (0.8707837245230915, array([93,  4,  4])),
 (0.8708625490311901, array([96,  4,  3])),
 (0.8712579185353593, array([95,  3,  4])),
 (0.8715698221827024, array([95,  3,  3])),
 (0.872130010851587, array([94,  2,  4])),
 (0.8725239177557818, array([94,  2,  3])),
 (0.8733358483560549, array([97,  4,  4])),
 (0.8737339452985942, array([97,  4,  3])),
 (0.8738465605805246, array([96,  3,  4])),
 (0.8740117897613403, array([93,  3,  4])),
 (0.8740584424362383, array([93,  3,  3])),
 (0.8743210352510705, array([96,  3,  3])),
 (0.8746005940559757, array([95,  2,  4])),
 (0.8751550658002649, array([95,  2,  3])),
 (0.8756004918948385, array([94,  1

In [7]:
import datetime 

time_start = datetime.datetime.now()

In [8]:
time_end = datetime.datetime.now()

In [11]:
type((time_end - time_start).total_seconds())

float